In [1]:
# ======================================================================
# ⚙️ 0. 라이브러리 임포트 및 분석 시작
# ======================================================================
# 이 코드를 실행하기 전, cmd에서 아래 라이브러리가 설치되었는지 확인해주세요.
# pip install bertopic pandas torch
# ======================================================================
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from ast import literal_eval
import os
import torch

# 1. 전처리된 데이터 불러오기
try:
    # notebooks 폴더의 상위 폴더(my_thesis)에 있는 data 폴더를 가리키도록 수정
    df = pd.read_csv('../data/processed_reviews.csv')
    df['tokens'] = df['tokens'].apply(literal_eval)
    print("----- 전처리된 데이터 불러오기 성공 -----")
except FileNotFoundError:
    print("오류: '../data/processed_reviews.csv' 파일을 찾을 수 없습니다. 02_preprocessing.ipynb를 먼저 실행했는지 확인하세요.")
    exit()

# 2. BERTopic 모델 학습을 위한 데이터 준비
documents = df['tokens'].apply(lambda tokens: ' '.join(tokens))
print("\n----- BERTopic 입력용 문서 샘플 -----")
print(documents.head())

# 3. BERTopic 모델 초기화 및 학습
print("\n----- BERTopic 모델 학습 시작 -----")

# GPU(CUDA) 사용 설정
# CUDA를 사용할 수 있으면 'cuda'를, 그렇지 않으면 'cpu'를 자동으로 선택합니다.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"----- 사용할 디바이스: {device.upper()} -----")

# SentenceTransformer 모델을 미리 GPU에 로드합니다.
embedding_model_name = "jhgan/ko-sbert-nli"
embedding_model = SentenceTransformer(embedding_model_name, device=device)

# BERTopic 모델에 GPU에 로드된 모델 '객체'를 직접 전달합니다.
topic_model = BERTopic(embedding_model=embedding_model,
                       verbose=True,
                       min_topic_size=3)

# 모델 학습
topics, probs = topic_model.fit_transform(documents)

# 4. 분석 결과 확인 및 저장
df['topic'] = topics

print("\n----- 토픽 할당 결과 -----")
print(df[['review', 'topic']].head(10))

topic_info = topic_model.get_topic_info()
print("\n----- 전체 토픽 정보 -----")
print(topic_info)

print("\n----- 토픽별 상위 키워드 -----")
for i in range(len(topic_info)):
    topic_num = topic_info.iloc[i]["Topic"]
    if topic_num == -1: continue # -1은 이상치(outlier) 토픽이므로 건너뜁니다.
    keywords = topic_model.get_topic(topic_num)
    print(f"Topic {topic_num}: {[keyword[0] for keyword in keywords]}")

# 5. 결과 저장
# 상위 폴더에 models 폴더를 생성하도록 수정
if not os.path.exists("../models"):
    os.makedirs("../models")

# 상위 폴더의 data 폴더에 저장하도록 수정
df.to_csv('../data/reviews_with_topics.csv', index=False, encoding='utf-8-sig')
# 상위 폴더의 models 폴더에 저장하도록 수정
topic_model.save("../models/bertopic_model", serialization="safetensors")

print(f"\n\n✅ 토픽 모델링이 완료되어 '../data/reviews_with_topics.csv' 파일과 '../models/bertopic_model'을 저장했습니다.")



----- 전처리된 데이터 불러오기 성공 -----

----- BERTopic 입력용 문서 샘플 -----
0    애정 오늘 다양 모찌리도후 품절 안쪽 자리 리모델링 안주 금액 최고입니댜 범계 맛집...
1               인기 모찌리도후 오늘 마지막 해물야끼 우동 처음 범계 술집 범계 맛집
2    인계동 범계 공휴일 저녁 웨이팅 안내 김치 카츠 모듬 꼬치 가격대 꼬치 종류 랜덤 ...
3                         범계 술집 범계 맛집 모임 장소 남편 데이트 분위기
4                       범계 술집 안주 음료 소주 재미 본관 별관 분위기 만족
Name: tokens, dtype: object

----- BERTopic 모델 학습 시작 -----
----- 사용할 디바이스: CPU -----


tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2025-09-25 12:45:44,441 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2025-09-25 12:45:47,224 - BERTopic - Embedding - Completed ✓
2025-09-25 12:45:47,229 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-25 12:45:55,385 - BERTopic - Dimensionality - Completed ✓
2025-09-25 12:45:55,386 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-25 12:45:55,392 - BERTopic - Cluster - Completed ✓
2025-09-25 12:45:55,395 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-25 12:45:55,409 - BERTopic - Representation - Completed ✓



----- 토픽 할당 결과 -----
                                              review  topic
0  언제나 애정하는 잔잔입니다:)\n오늘은\n다양하게 먹어봤어오! 모찌리도후는\n먹고싶...      0
1  인기많은 모찌리도후 오늘은 마지막으로 겟 했습니다:)\n해물야끼우동 처음 먹어보는데...      0
2  항상 인계동에서만 가봤다가 범계는 처음 가봤어요. 공휴일 저녁이라 역시 웨이팅이 있...      1
3  #범계술집#범계맛집#범계모임장소\n남편과 데이트 할때면 무조건 오는 잔잔~^^\n맛...      0
4  범계술집하면 믿고 먹는 잔잔이죠 !! 안주 뭘 시켜도 너무 맛있고 하이볼 음료 삼아...      0
5  매번 지나가면서 와야지 와야지 하다 드디어 처음 와봣는데 생맥주 너무 맛잇네요~ 매...      0
6  저번에 맛있어서 또 방문했어요ㅎㅎ 범계 술집 맛집 답게 사람도 많고 음식도 역시 맛...     -1
7  이자카야 잔잔 정말 맛있습니다. 일본식 안주 좋아하시면 강추드립니다. 분위기도 아늑...      2
8  범계 술집 잔잔!!!\n검색하고 왔는데 분위기는 기본 친절한 서비스~~\n넘넘 기분...      2
9  범계역에서 동료들과 1차 끝내고 2차 장소 찾다가 들어왔어요. 최근 일본 여행을 많...      2

----- 전체 토픽 정보 -----
   Topic  Count              Name  \
0     -1     13    -1_음식_오늘_저번_만족   
1      0     27     0_술집_범계_안주_맛집   
2      1     22  1_볶음밥_일본_웨이팅_파스타   
3      2     20  2_이자카야_이태원_범계_사장   
4      3     14     3_친절_직원_감사_음식   
5      4      8   4_구이_테바사키_고기_차원   
6      5      8     5_만족_특별_생각_안주   
